In [2]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import os 
from saved_tracks_extractor import get_audio_features_and_analysis, get_genres
from dotenv import load_dotenv

# Configura le tue credenziali Spotify
load_dotenv()
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')
scope = "user-library-read playlist-modify-public"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
#                                                client_secret=client_secret,
#                                                redirect_uri=redirect_uri,
#                                                scope=scope))


sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [ ]:
target_user_id = '31ighcy6cpa3jsteh7imcasje2ea'

In [ ]:
# Initialize an empty list to store the results
track_data = {}

# Set the batch size (max 50 for Spotify API)
batch_size = 20

# Get the total number of saved tracks
total_tracks = sp.current_user_saved_tracks(limit=1)['total']
print(f"Total tracks: {total_tracks}")

total_tracks = 60
# Loop through batches of saved tracks
for offset in range(0, total_tracks, batch_size):
    tracks = sp.current_user_saved_tracks(limit=batch_size, offset=offset)
    track_ids = [item['track']['id'] for item in tracks['items']]
    print(f"Track IDs: {track_ids}")    
    # Apply the get_audio_features_and_analysis function
    track_data_old = get_genres(track_ids, sp)
    # Append the result to the list
    if offset==0:
        track_data=track_data_old
    else:
        for key in track_data:
            track_data[key]+=track_data_old[key]
    
print(track_data.keys())
# Print or process the results as needed
# for result in results:
#     print("---")
#     print(json.dumps(result, indent=4))

In [ ]:
print(track_data['genres'])
print(len(track_data['id']))

In [ ]:
genre_dict= dict(zip(track_data['id'],track_data['genres']))
print(genre_dict)


In [ ]:
genre_list=[]
for song in genre_dict:
    for genre in genre_dict[song]:
        if genre not in genre_list:
            genre_list.append(genre)

genre_list.append('no genre')
print(genre_list)

In [ ]:
playlist_dict={}
no_genre_songs=[]
for genre in genre_list:
    song_list=[]
    for song in genre_dict:
        if genre in genre_dict[song] and song not in song_list:
            song_list.append(song)
        if not genre_dict[song] and song not in no_genre_songs:
            no_genre_songs.append(song)

    playlist_dict[genre]=song_list

playlist_dict['no genre']=no_genre_songs


print(len(list(playlist_dict.keys())))
print(playlist_dict)
print(len(playlist_dict['no genre']))


In [ ]:
batch_size=50
user_playlists=[]

for offset in range(0, 9000, batch_size):
    user_playlists_batch=sp.user_playlists(user=target_user_id,limit=batch_size,offset=offset)
    user_playlists += user_playlists_batch["items"]
    print(len(user_playlists_batch["items"]))
    if len(user_playlists_batch["items"]) < batch_size:
        break

In [ ]:
uplaylist_test=sp.user_playlists(user=target_user_id)
print(type(user_playlists_batch['items']))
print(len(uplaylist_test))
print(type(uplaylist_test))
print(len(uplaylist_test["items"]))
#print(json.dumps(uplaylist_test,indent=2))
print(len(user_playlists))
print(json.dumps(user_playlists[0],indent=2))
#ignore this cell

In [ ]:
for key in list(playlist_dict.keys()):
    for playlist in user_playlists:
        if playlist['name'] == key:
            x=sp.user_playlist_unfollow(user=target_user_id,playlist_id=playlist['id']) 
            #unfollow cleans the account from any playlist with the same name

    playlist_created=sp.user_playlist_create(user=target_user_id,name=key,public=True,collaborative=False,description='')
    #creates the playlist for the name 'key'

    for songs in playlist_dict[key]:
        playlist_songlist=sp.user_playlist_add_tracks(user=target_user_id, playlist_id=playlist_created['id'], tracks=playlist_dict[key], position=None)
    #adds songs saved in the genre-songs dictionary in the playlist created before